# Accessing the GA Aquatic ARD database

This notebook provides access to the test database of Aquatic ARD data produced for a selection of Landsat 8 OLI and Sentinel 2 MSI scenes for inland water bodies in Australia

See the accompanying README.rst for details of how to connect to the database  and database coverage.


## Setting up the modules and tools

In [ ]:
import numpy
import sys
import matplotlib.pyplot as plt
import datacube
#sys.path.append('../Scripts')
sys.path.insert(1, '../Tools/')
from sungc import deglint, visualise
from sungc import rasterio_funcs as rf
#from dea_bandindices import calculate_indices
import dea_tools.plotting  # or some other submodule
import dea_tools.datahandling

## Selection a study region

Specify the study region and check the extent on the map display

A region code can also be used for Sentinel 2 granules (eg. 55HFB) or Landsat Path/Row (eg. 091084), which can be specified in the query below in place of lat/lon.

In [ ]:
lat_range = (-35.003, -35.031)
lon_range = (148.627, 148.655)

In [ ]:
dea_tools.plotting.display_map(x=lon_range, y=lat_range)

## Connect to the Aquatic ARD datacube

In [ ]:
dc = datacube.Datacube(env="water-atcor", config="/g/data/r78/dea-atmospheric-correction-over-water/datacube.conf")

Using Sentinel-2a as an example, you can view the product and measurements available

In [ ]:
product = "ga_s2am_aard_2"

measurements = dc.list_measurements()
measurements.loc[product]

The `ga_s2am_aard_2` product is comprised of the following sub-products:

* lmbskyg `lambertian sky glint corrected`
* oa `observation attributes`

## Set up a data query and load the data

In [ ]:
query = {
    'product': 'ga_s2am_aard_2',
    #'product': 'ga_s2bm_aard_2', #option to search Sentinel 2b
    #'product': 'ga_ls8c_aard_3', #option to search Landsat 8 
    #'region_code':'55HBT', #option to use Sentinel 2 granule ID ot LS path/row
    'x':lon_range,
    'y':lat_range,
    'time': ('2017-04-17', '2017-04-19'),
    'output_crs': 'epsg:3577',
    'resolution': (-20, 20)
}

In [ ]:
ds = dc.load(**query)
ds = ds.where(ds.oa_fmask.isin([1, 5])) #retain only valid water pixels using fmask, masking clouds

### View what the data looks like as an xarray

In [ ]:
print(ds)

### Plot as an RGB image using DEA tools

In [ ]:
dea_tools.plotting.rgb(ds,
                       index=0,
                      bands=['lmbskyg_red','lmbskyg_green','lmbskyg_blue'],
                      percentile_stretch=['0.6', '0.98'])

### *******Another way of loading data with enhanced functionality using DEA tools DOES NOT yet work with Deglinting code*******

In [ ]:
ds = dea_tools.datahandling.load_ard(dc=dc,
              products=['ga_s2am_aard_2'],
              x=lon_range,
              y=lat_range,
              time=('2017-04-17', '2017-04-19'),
              fmask_categories=['water'],
              measurements=['lmbskyg_coastal_aerosol', 'lmbskyg_blue', 'lmbskyg_green', 'lmbskyg_red', 'lmbskyg_red_edge_1', 'lmbskyg_red_edge_2','lmbskyg_nir_2'],
              output_crs='EPSG:3577',
              resolution=(-20, 20),
              group_by='solar_day')

# Deglinting Modules

This section demonstrates the different deglinting methods available witin the sungc module:

* Hedley et al., 2005 (correlation approach)
* Cox and Munk 1954 (statistical/geometry approach)
* NIR/SWIR subtraction (e.g. Dierssen et al., 2015)

The sunglint module can be installed by cloning repository from *https://github.com/GeoscienceAustralia/sun-glint-correction* and following the installation instructions located in the README.md file

In [ ]:
g = deglint.GlintCorrX(ds)

Specify the bands to be used in creating a quicklook RGB

In [ ]:
rgb_bands = ['lmbskyg_red', 'lmbskyg_green', 'lmbskyg_blue']

vis_bands=['lmbskyg_coastal_aerosol', 'lmbskyg_blue', 'lmbskyg_green', 'lmbskyg_red', 'lmbskyg_red_edge_1', 'lmbskyg_red_edge_2']


## Hedley 2005
The Hedley method requires a Region Of Interest to be selected (or provided) that contains both deep water and sun glint affected pixels.

In [ ]:
%matplotlib
hedley_xarr = g.hedley_2005(
    rgb_varlist=rgb_bands,
    vis_bands=vis_bands,
    corr_band='lmbskyg_nir_2',
)

## Cox and Munk 1954

In [ ]:
cm_xarr = g.cox_munk(
    vis_bands=vis_bands,
    wind_speed=5,
    vzen_band="oa_satellite_view",
    szen_band="oa_solar_zenith",
    razi_band="oa_relative_azimuth",
)

## Glint subtraction

In [ ]:
subtract_xarr = g.glint_subtraction(
    vis_bands=vis_bands,
    corr_band="lmbskyg_nir_2",
)

## Comparison

In [ ]:
%matplotlib inline
annotate_list = [
        "Initial",
        "Cox & Munk (1954)",
        "Hedley (2005)",
        "NIR subtraction",
    ]

fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(14, 13))
for t in range(ds.dims["time"]):
    master_ds = [
        ds.isel(time=t),
        cm_xarr.isel(time=t),
        hedley_xarr.isel(time=t),
        subtract_xarr.isel(time=t),
    ]

    for i, ds_i in enumerate(master_ds):
        rix, cix = numpy.unravel_index(i, (2, 2))

        refl_im = numpy.array(
            [
                ds_i.variables[rgb_bands[0]].values,  # R
                ds_i.variables[rgb_bands[1]].values,  # G
                ds_i.variables[rgb_bands[2]].values,  # B
            ],
            order="C"
        )

        rgb_im = visualise.seadas_style_rgb(refl_im, [0, 1, 2], 10000.0, True)
        
        axes[rix, cix].imshow(rgb_im, interpolation="None")
        axes[rix, cix].axis("off")
        axes[rix, cix].annotate(
            annotate_list[i],
            xy=(0.05, 0.94),
            xycoords="axes fraction",
            fontsize=10,
            color="w",
        )

    fig.subplots_adjust(
        left=0.01, right=0.99, bottom=0.01, top=0.99, wspace=0.01
    )